# 데이터 작업하기

torchvision dataset 중 fashionMNIST data를 test data와 training data로 불러온다.

다음 위와 같은 dataset을 dataloader의 인자로 전달한다. 사실 정확하게 무슨 의미인 지는 모르겠지만 내 생각에는 dataloader로 dataset을 전달함으로써 해당 데이터들을 학습에 용이하게끔 shuffle, batch, sampling 등이 가능하게 만드는 것 같다... maybe...


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:03<00:00, 7478117.70it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 127979.43it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:04<00:00, 1063314.85it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5235760.67it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# 모델 만들기

먼저 어떤 device(보통 cpu나 gpu)를 이용해 학습을 할 것인 지 정한 뒤, 모델을 정의한다.
_init_을 이용해 학습을 진행할 모델의 layer를 설정한다. 모델을 정의하는 구체적인 방법은 뒤에 나올 예정

In [5]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# 모델 매개변수 최적화하기

모델 학습에 필요한 loss function과 optimizer를 정의

이전 단계에서 생성한 dataloader를 이용해서 trainig과 test를 진행한다.

training 이후에는 test를 진행하여 성능을 확인할 수 있다.
(아래 코드에서는 epoch 5번으로 진행)

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312913  [   64/60000]
loss: 2.300011  [ 6464/60000]
loss: 2.271227  [12864/60000]
loss: 2.258134  [19264/60000]
loss: 2.250363  [25664/60000]
loss: 2.209229  [32064/60000]
loss: 2.232571  [38464/60000]
loss: 2.202788  [44864/60000]
loss: 2.195877  [51264/60000]
loss: 2.144537  [57664/60000]
Test Error: 
 Accuracy: 24.9%, Avg loss: 2.148339 

Epoch 2
-------------------------------
loss: 2.167590  [   64/60000]
loss: 2.163565  [ 6464/60000]
loss: 2.096827  [12864/60000]
loss: 2.111209  [19264/60000]
loss: 2.059567  [25664/60000]
loss: 1.992157  [32064/60000]
loss: 2.035280  [38464/60000]
loss: 1.962467  [44864/60000]
loss: 1.967985  [51264/60000]
loss: 1.876474  [57664/60000]
Test Error: 
 Accuracy: 53.9%, Avg loss: 1.882629 

Epoch 3
-------------------------------
loss: 1.925225  [   64/60000]
loss: 1.895024  [ 6464/60000]
loss: 1.775904  [12864/60000]
loss: 1.818434  [19264/60000]
loss: 1.692001  [25664/60000]
loss: 1.651503  [32064/600

# 모델 저장하기&불러오기

일반적으로 모델을 저장할 때는 직렬화(serialization)를 해서 저장한다. 이후 저장한 모델을 다시 불러올 때는 역직렬화(de-serialization) 과정을 통해 원래 객체로 복원한다. 불러온 모델을 이용하여 학습을 진행할 수 있다.

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
